In [1]:
import sys, os
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

# Cell 1: Xử lý missing
import numpy as np
# Giả định src.data_processing chỉ dùng numpy và các thư viện cơ bản
from src.data_processing import convert_columns, load_airbnb, save_cols_dict_to_csv, load_airbnb_processed
# Cấm import csv

# --- 1. Tải và Chuyển đổi Dữ liệu ---
header, rows_raw = load_airbnb()
cols_dict = convert_columns(rows_raw, header)

# Phân loại cột
numeric_cols = [name for name, d in cols_dict.items() 
                if d["is_numeric"] and name not in ["id", "host_id"]]

categorical_cols = [name for name, d in cols_dict.items() 
                    if not d["is_numeric"] or name in ["id", "host_id"]]

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

# --- 2. Xử lý Missing Values (TRIỆT ĐỂ) ---

# --- Numeric missing ---
for name in numeric_cols:
    col = cols_dict[name]["values"]
    
    # Ép kiểu dữ liệu sang float64 trước để np.isnan hoạt động chính xác
    if col.dtype != np.float64:
        try:
            col = np.asarray(col, dtype=np.float64)
        except ValueError:
            print(f"BỎ QUA: Cột '{name}' không thể chuyển thành float.")
            continue
            
    nan_mask = np.isnan(col)
    
    if np.any(nan_mask):
        nan_count = np.sum(nan_mask)
        
        if name == "reviews_per_month":
            # Logic 1: reviews_per_month (Missing/NaN) = 0.0
            col[nan_mask] = 0.0
            print(f"Xử lý missing cho '{name}': Thay thế {nan_count:,} NaN bằng 0.0.")
        
        elif name in ["latitude", "longitude", "price", "minimum_nights", 
                      "number_of_reviews", "calculated_host_listings_count", 
                      "availability_365"]:
            # Logic 2: Các cột số quan trọng khác (bao gồm tọa độ) dùng Median
            median_val = np.nanmedian(col)
            col[nan_mask] = median_val
            print(f"Xử lý missing cho '{name}': Thay thế {nan_count:,} NaN bằng Median ({median_val:,.2f}).")

        # Cập nhật lại cột đã xử lý
        cols_dict[name]["values"] = col

# --- Categorical missing ---
for name in categorical_cols:
    col = cols_dict[name]["values"]
    missing_mask = (col == '')
    
    if np.any(missing_mask):
        col[missing_mask] = 'Unknown'
        cols_dict[name]["values"] = col

save_path = os.path.join("..\\data", "processed", "AB_NYC_2019_preprocessed.csv")
save_cols_dict_to_csv(cols_dict, header, save_path) 

print(f"Saved featured file to {save_path}.")

Numeric columns: ['latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
Categorical columns: ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review']
Xử lý missing cho 'reviews_per_month': Thay thế 10,052 NaN bằng 0.0.
--- BẮT ĐẦU GHI FILE VỚI ĐỊNH DẠNG CSV CHUẨN (CÓ DẤU NGOẶC KÉP) ---

--- HOÀN TẤT PREPROCESSING (Missing Handled & CSV Formatted) ---
Saved preprocessed file to ..\data\processed\AB_NYC_2019_preprocessed.csv.
